In [2]:
import matplotlib.pyplot as plt
import json
import glob
import re
import pandas as pd
import numpy as np

import qutip
from CoupledQuantumSystems.drive import *
from scipy.optimize import minimize
from CoupledQuantumSystems.IFQ import gfIFQ
from CoupledQuantumSystems.evo import ODEsolve_and_post_process

import multiprocessing
multiprocessing.set_start_method('spawn')

In [3]:
EJ = 3
EJoverEC = 6
EJoverEL = 25
EC = EJ / EJoverEC
EL = EJ / EJoverEL
n_lvls = 80
n_lvls_exp = 60
qbt = gfIFQ(EJ = EJ,EC = EC, EL = EL, flux = 0,truncated_dim=n_lvls)
e_ops = [qutip.basis(qbt.truncated_dim, i)*qutip.basis(qbt.truncated_dim, i).dag() for i in range(n_lvls_exp)]

state_0_dressed = qutip.basis(qbt.truncated_dim, 1)
state_1_dressed = qutip.basis(qbt.truncated_dim, 2)

initial_states  = [
                state_0_dressed,
                state_1_dressed,
                ]

def do_experiment(amp, w_d, t_duration,plot=False):
    t_tot = t_duration
    tlist = np.linspace(0, t_tot, 51)

    results = qbt.run_qutip_mesolve_parrallel(
                initial_states,
                [tlist],
                drive_terms=[[
                            DriveTerm(
                                driven_op=qutip.Qobj(
                                    qbt.fluxonium.n_operator(energy_esys=True)),
                                pulse_shape_func=sin_squared_pulse_with_modulation,
                                pulse_id='pi',
                                pulse_shape_args={
                                    "w_d": w_d,    # No extra 2pi factor
                                    "amp": amp,    # No extra 2pi factor
                                    't_duration': t_duration,
                                },
                            )
                        ]],
                c_ops=None,
                e_ops=[e_ops],
    )[0]
    if plot:
        fig, axes = plt.subplots(1,2,figsize=(8,4))
        for exp_idx in range(len(e_ops)):
            axes[0].plot(tlist, results[0].expect[exp_idx],label= f"|{exp_idx}>")
        axes[0].set_yscale('log')

        for exp_idx in range(len(e_ops)):
            axes[1].plot(tlist, results[1].expect[exp_idx],label= f"|{exp_idx}>")
        axes[1].set_yscale('log')

    one_minus_pop2 = np.abs(1- results[0].expect[2][-1])
    one_minus_pop0 = np.abs(1- results[1].expect[1][-1])
    res = one_minus_pop2 + one_minus_pop0
    return res

In [4]:
# load unique_t_tot
with open("results_unique.json", "r") as f:
    results = [json.loads(line) for line in f]
unique_t_tot = {}
for result in results:
    # get rid of frequency
    result["w_d"] = 0
    if result["t_tot"] not in unique_t_tot:
        unique_t_tot[result["t_tot"]] = result
        if "fun" not in result:
            fun = do_experiment(result["amp"], result["w_d"], result["t_tot"]*0.85, result["t_tot"]*0.15)
            result["fun"] = fun
    else: # (result["t_tot"] in unique_t_tot)
        if "fun" in result:
            if result["fun"] < unique_t_tot[result["t_tot"]]["fun"]:
                unique_t_tot[result["t_tot"]] = result
        else:
            fun = do_experiment(result["amp"], result["w_d"], result["t_tot"]*0.85, result["t_tot"]*0.15)
            result["fun"] = fun
            if result["fun"] < unique_t_tot[result["t_tot"]]["fun"]:
                unique_t_tot[result["t_tot"]] = result    


# Store to new json
with open("results_sin^2_no_frequency_unique.json", "w") as f:
    for result in unique_t_tot.values():
        json.dump(result, f)
        f.write("\n")

In [6]:
from IPython.display import clear_output

for t_tot in unique_t_tot:

    def objective_with_grad(x):
        amp = x[0]
        w_d = 0
        epsilon = 1e-3

        tlist = np.linspace(0, t_tot, 51)

        delta_amp = abs(amp*epsilon)
        amp_forward = amp + delta_amp
        amp_backward = amp - delta_amp

        results_five_by_two = qbt.run_qutip_mesolve_parrallel(
                    initial_states,
                    [tlist for _ in range(3)],
                    drive_terms=[[ DriveTerm(
                                    driven_op=qutip.Qobj(
                                        qbt.fluxonium.n_operator(energy_esys=True)),
                                    pulse_shape_func=sin_squared_pulse_with_modulation,
                                    pulse_id='pi',
                                    pulse_shape_args={
                                        "w_d": w_d,
                                        "amp": amp,
                                        't_duration': t_tot,
                                    },)],
                            [DriveTerm(
                                    driven_op=qutip.Qobj(
                                        qbt.fluxonium.n_operator(energy_esys=True)),
                                    pulse_shape_func=sin_squared_pulse_with_modulation,
                                    pulse_id='pi',
                                    pulse_shape_args={
                                        "w_d": w_d,
                                        "amp": amp_forward,
                                        't_duration': t_tot,
                                    },)],
                            [DriveTerm(
                                    driven_op=qutip.Qobj(
                                        qbt.fluxonium.n_operator(energy_esys=True)),
                                    pulse_shape_func=sin_squared_pulse_with_modulation,
                                    pulse_id='pi',
                                    pulse_shape_args={
                                        "w_d": w_d,
                                        "amp": amp_backward,
                                        't_duration': t_tot,
                                    },)],
                            ],
                    c_ops=None,
                    e_ops=[e_ops for _ in range(3)],
                    )

        grad = np.zeros_like(x, dtype=float)

        obj_val = np.zeros((5,))
        for i, results in enumerate(results_five_by_two):
            one_minus_pop2 = np.abs(1- results[0].expect[2][-1])
            one_minus_pop0 = np.abs(1- results[1].expect[1][-1])
            res = one_minus_pop2 + one_minus_pop0
            obj_val[i] = res
        # print(obj_val)
        grad[0] = (obj_val[2] - obj_val[1]) / (2 * delta_amp)  
        print(f"t_tot: {t_tot}, fun: {obj_val[0]}, amp: {amp}")
        return obj_val[0], -grad

    def manual_nadam(x0, learning_rates, num_iterations=100, stop_threshold=1e-4, beta1=0.9, beta2=0.999, epsilon=1e-8):
        # Initialize parameters
        x = np.array(x0, dtype=np.float32)
        m = np.zeros_like(x)  # First moment (mean of gradients)
        v = np.zeros_like(x)  # Second moment (uncentered variance of gradients)
        t = 0  # Time step

        prev_loss = np.inf  # Store the loss value from the previous iteration
        for i in range(num_iterations):
            t += 1
            # Get the objective value and gradient
            obj_val, grad = objective_with_grad(x)

            # Update biased first and second moment estimates
            m = beta1 * m + (1 - beta1) * grad
            v = beta2 * v + (1 - beta2) * grad**2

            # Compute bias-corrected first and second moment estimates
            m_hat = m / (1 - beta1**t)
            v_hat = v / (1 - beta2**t)

            # Apply Nesterov momentum
            m_nesterov = beta1 * m_hat + (1 - beta1) * grad

            # Update parameters with different learning rates
            x = x - learning_rates * m_nesterov / (np.sqrt(v_hat) + epsilon)

            # Check if the gradient is small enough to stop (optional condition)
            grad_norm = np.linalg.norm(grad)
            if grad_norm < stop_threshold:
                print(f"Stopping early: Gradient norm {grad_norm} is below threshold.")
                break

            # Check if the loss is changing too slowly (convergence)
            loss_diff = np.abs(prev_loss - obj_val)
            if loss_diff < stop_threshold:
                print(f"Stopping early: Loss difference {loss_diff} is below threshold.")
                break

            # Update the previous loss
            prev_loss = obj_val

        return x, prev_loss  # Final optimized values
    
    dictionary = unique_t_tot[t_tot]
    fun_old = dictionary["fun"]
    x0 = [dictionary["amp"]]
    
    optimized_params, prev_loss = manual_nadam(x0, np.array([0.1]), num_iterations=100)
    last_amp, = optimized_params
    
    if prev_loss < fun_old:
        #Append result to file
        with open("results_sin^2_no_frequency_unique.json", "a") as f:
            json.dump({
                "t_tot": float(t_tot),
                "amp": float(last_amp),
                "w_d": float(0),
                'fun': float(prev_loss)
            }, f)
            f.write("\n")

        #print result
        # clear_output(wait=True)
        print(f"t_tot: {t_tot}, fun: {prev_loss}, amp: {last_amp}")

t_tot: 171.5, fun: 0.5472426116574594, amp: 0.5378928780555725
t_tot: 171.5, fun: 0.3997741508332773, amp: 0.637892877602633
t_tot: 171.5, fun: 0.39576003589310915, amp: 0.7199500149681326
t_tot: 171.5, fun: 0.43104083120960235, amp: 0.7595494505826734
t_tot: 171.5, fun: 0.4313245613342931, amp: 0.7597896183762349
t_tot: 171.5, fun: 0.40879818445397453, amp: 0.7378077018144309
t_tot: 171.5, fun: 0.389271210368763, amp: 0.7068148796692658
t_tot: 171.5, fun: 0.3845607386215266, amp: 0.6767691238276138
t_tot: 171.5, fun: 0.3900168315243555, amp: 0.6551998864126996
t_tot: 171.5, fun: 0.39474435543623987, amp: 0.6457614969117809
t_tot: 171.5, fun: 0.39368111893792235, amp: 0.6476561707142922
t_tot: 171.5, fun: 0.3891505636218845, amp: 0.6573211994030306
t_tot: 171.5, fun: 0.3854008750781568, amp: 0.6703889892681868
t_tot: 171.5, fun: 0.38436350113264284, amp: 0.6828646750132695
t_tot: 171.5, fun: 0.3851528079205385, amp: 0.6918876746062146
t_tot: 171.5, fun: 0.3859708617005789, amp: 0.69614

KeyboardInterrupt: 